# 해당 파일 설명
이 코드를 모두 실행하고 나면, /converted 경로에 다음의 파일이 생성됩니다.

1. plant_all_weather_feature_add_cluster.csv
2. plant_test_all_weather_feature_add_cluster.csv
3. model/kmeans_cluster_model.pkl

## 추가 설명
위 파일은, 4_plant_weather_combine.ipynb 에서 만들어진 

1. plant_all_weather_feature.csv
2. plant_test_all_weather_feature.csv

두 파일을 기반으로 클러스터링 된 변수를 추가하는 파일입니다. 클러스터는, 각 해당 일자의 그래프의 개형을 기반으로 분류된 클래스를 넣어주도록 합니다.

# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
#BASE_WORKING_DIR = "/content/gdrive/My Drive/Colab Notebooks/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 backup_0.96model
data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
lastplus_th_10.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_cond_all_plus_.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.c

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Get the Data

## 1. 사용하는 함수


In [ ]:
import pickle
from sklearn.externals import joblib
cluster_trend_dict = dict()
def trend_fn(where, target, k=5):
  global cluster_trend_dict
  temp_trend = pd.DataFrame(columns=['date','0-4', '4-8','8-12','12-16','16-20','20-24'])

  date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(where, target, date)
    temp_date = pd.DataFrame(columns=['date','0-4', '4-8','8-12','12-16','16-20','20-24'])
    temp_date.loc[0] = 0
    temp_date['date'] = date
    for time in range(0, 24, 4):
      start_time = time * 60
      end_time = (time + 4) * 60
      # if end_time == 240:
      #   end_time = 0
      key = (where, target, date)
      ipo = ipo_dict[key]
      start = spi.splev(start_time, ipo)
      end = spi.splev(end_time, ipo)
      temp_date['{}-{}'.format(time, time + 4)] = start - end
    temp_trend = temp_trend.append(temp_date)  # append를 사용하면 concat이랑 같은 효과임
    date += timedelta(days=1)

  # 각 일에 대한 interpolation 함수를 대변하는 feature를 기반으로 클러스터링을 진행한다.
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(temp_trend[['0-4', '4-8','8-12','12-16','16-20','20-24']])
  key = (where, target)
  cluster_trend_dict[key] = copy.deepcopy(kmeans)
  temp_trend[where+'-'+target+'-trend'] = kmeans.predict(temp_trend[['0-4', '4-8','8-12','12-16','16-20','20-24']])
  return temp_trend

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.externals import joblib
cluster_trend_dict = dict()
def trend_fnx(where, target, k=5):  ############## 사용할때 fn뒤의 x를빼고 쓸것
  global cluster_trend_dict
  timeList = []
  for i in range(23):     # 23시까지 하는 이유는 보간으로 인해 24시의 값이 불안정 하기 때문
    timeList.append(f'{i}-{i+1}')
  for i in range(22):     # 2차 기울기
    timeList.append(f'{i}-{i+1}-second')
  timeList.insert(0, "min")
  timeList.insert(0, "max")
  timeList.insert(0, "max-min-avg-slope")
  timeList.insert(0, "date")
  timeList = list(map(str, timeList))
  # print("1 : ", timeList)


  temp_trend = pd.DataFrame(columns=timeList)

  date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(where, target, date)
    
    temp_date = pd.DataFrame(columns=timeList)
    temp_date.loc[0] = 0
    temp_date['date'] = date

    # 보간함수 불러오기
    key = (where, target, date)
    ipo = ipo_dict[key]

    # Max, Min column 넣어주기
    x = np.linspace(0, 24 * 60, 1000)
    y = spi.splev(x, ipo)
    min_arg = x[np.argmin(y)]
    max_arg = x[np.argmax(y)]
    temp_date["min"] = np.min(y)
    temp_date["max"] = np.max(y)
    temp_date["max-min-avg-slope"] = (temp_date.loc[0, "max"] - temp_date.loc[0, "min"])/(max_arg-min_arg)
    # print("2 : ", temp_date)
    
    # 시간대에 맞는 값 넣어주기
    for hour in range(23):
      time0 = hour * 60
      time1 = (hour+1) * 60
      value0 = spi.splev(time0, ipo)
      value1 = spi.splev(time1, ipo)
      temp_date[f'{hour}-{hour+1}'] = (value1-value0)/60
    # print("3 : ", temp_date)

    for hour in range(22):
      temp_date[f'{hour}-{hour+1}-second'] = (temp_date.loc[0, f'{hour}-{hour+1}']-temp_date.loc[0, f'{hour+1}-{hour+2}'])/60

    # 만들어진 df 합치기
    temp_trend = temp_trend.append(temp_date)
    date += timedelta(days=1)
  
  # print("3 : ", temp_trend)

  # 만들어진 temp_trend 를 가지고, PCA를 수행한다.
  x = temp_trend.drop(['date', "min", "max", "max-min-avg-slope"], axis=1).values # 독립변인들의 value값만 추출
  x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장
  
  pca = PCA(n_components=45) # 주성분을 몇개로 할지 결정
  printcipalComponents = pca.fit_transform(x)

  # eigen value를 통해, 몇개의 축을 선정할 것인지 판단하자.
  print('explained variance ratio :', pca.explained_variance_ratio_)

  # 주성분으로 이루어진 데이터 프레임 구성
  pcList = []
  for pc in range(1, 21):
    pcList.append(f'pc{pc}')
  pca_df = pd.DataFrame(data= printcipalComponents[:,:20], columns=pcList)
  print("4 : ", pca_df)

  # index 초기화
  temp_trend = temp_trend.reset_index()
  pca_df = pca_df.reset_index()

  # df 합치기
  temp_trend = pd.merge(temp_trend, pca_df, left_index=True, right_index=True, how='left')
  temp_trend = temp_trend.drop(["index_x", "index_y"], axis=1)
  print("5 : " ,temp_trend)

  # clustering을 위한 df
  # a = pcList.append("max-min-avg-slope")
  cluster_df = temp_trend[["pc1", "pc2", "pc3", "pc4", "pc5", "pc6", "pc7", "pc8", "pc9", "pc10", "pc11", "pc12", "pc13", "pc14", "pc15", "pc16", "pc17", "pc18", "pc19", "pc20", "max-min-avg-slope"]]
  print("6 : " ,cluster_df)

  kmeans = KMeans(n_clusters=k)
  kmeans.fit(cluster_df)
  key = (where, target)
  cluster_trend_dict[key] = copy.deepcopy(kmeans)
  temp_trend[where+'-'+target+'-trend'] = kmeans.predict(cluster_df)
  return temp_trend
  


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.externals import joblib
cluster_point_dict = dict()
def point_fn(where, target, k=5):
  global cluster_point_dict
  timeList = list(range(24))
  timeList.insert(0, "min")
  timeList.insert(0, "max")
  timeList.insert(0, "date")
  timeList = list(map(str, timeList))

  temp_point = pd.DataFrame(columns=timeList)

  date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(where, target, date)
    
    temp_date = pd.DataFrame(columns=timeList)
    temp_date.loc[0] = 0
    temp_date['date'] = date

    # 보간함수 불러오기
    key = (where, target, date)
    ipo = ipo_dict[key]

    # Max, Min column 넣어주기
    x = np.linspace(0, 24 * 60, 1000)
    y = spi.splev(x, ipo)
    temp_date["min"] = np.min(y)
    temp_date["max"] = np.max(y)
    
    # 시간대에 맞는 값 넣어주기
    for hour in range(24):
      time = hour * 60
      value = spi.splev(time, ipo)
      temp_date[f'{hour}'] = value
    
    # 만들어진 df 합치기
    temp_point = temp_point.append(temp_date)
    date += timedelta(days=1)

  # 만들어진 temp_point 를 가지고, PCA를 수행한다.
  x = temp_point.drop(['date', "min", "max"], axis=1).values # 독립변인들의 value값만 추출
  x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장
  
  pca = PCA(n_components=24) # 주성분을 몇개로 할지 결정
  printcipalComponents = pca.fit_transform(x)

  # eigen value를 통해, 몇개의 축을 선정할 것인지 판단하자.
  print('explained variance ratio :', pca.explained_variance_ratio_)

  # 주성분으로 이루어진 데이터 프레임 구성
  pca_df = pd.DataFrame(data= printcipalComponents[:,:2], columns=["pc1", "pc2"])

  # index 초기화
  temp_point = temp_point.reset_index()
  pca_df = pca_df.reset_index()

  # df 합치기
  temp_point = pd.merge(temp_point, pca_df, left_index=True, right_index=True, how='left')
  temp_point = temp_point.drop(["index_x", "index_y"], axis=1)
  # print(temp_point)

  # clustering을 위한 df
  cluster_df = temp_point[["pc1", "pc2", "min", "max"]]
  # print(cluster_df)

  kmeans = KMeans(n_clusters=k)
  kmeans.fit(cluster_df)
  key = (where, target)
  cluster_point_dict[key] = copy.deepcopy(kmeans)
  temp_point[where+'-'+target+'-point'] = kmeans.predict(cluster_df)
  return temp_point
  


## 2. 데이터 불러오기

In [ ]:
# plant + weather 추가한 전체 데이터 셋
# date를 datetime으로 변경하기
data_train = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature.csv"))
data_test = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_test_all_weather_feature.csv"))

# 'Unnamed: 0' 제거하기
data_train.drop(['Unnamed: 0'], axis='columns', inplace=True)
data_test.drop(['Unnamed: 0'], axis='columns', inplace=True)

# date -> datetime 형식으로 변경
data_train['date'] = data_train['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date())
data_test['date'] = data_test['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date())

In [ ]:
data_train

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0


In [ ]:
data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0


In [ ]:
# weather data에 필요한 함수 정보가 적혀있는 설명서를 load한다.
# interpolation dictionary 설명서를 load 한다.
IPO_DICT_READ_ME = pd.read_csv(os.path.join(MODEL_PATH, "IPO_DICT_READ_ME.csv"))
IPO_DICT_READ_ME = IPO_DICT_READ_ME.fillna(" ")
IPO_DICT_READ_ME

,WHERE :,dangjin,sinpyeong,seosan,songsan,songak
0,TARGET :,ta,ta,ta,ta,ta
1,,hm,ws,hm,hm,hm
2,,ws,rn,ws,ws,ws
3,,rn,lp,rn,rn,rn
4,,,ap,dew,,
5,,,,sp,,
6,,,,lp,,
7,,,,ap,,
8,,,,sr,,
9,,,,ss,,


In [ ]:
# interpolation dictionary 를 load 한다.
import pickle
ipo_dict = joblib.load(os.path.join(MODEL_PATH, "ipo_dict.pkl"))
ipo_dict.keys()

dict_keys([('dangjin', 'ta', datetime.date(2016, 4, 1)), ('dangjin', 'ta', datetime.date(2016, 4, 2)), ('dangjin', 'ta', datetime.date(2016, 4, 3)), ('dangjin', 'ta', datetime.date(2016, 4, 4)), ('dangjin', 'ta', datetime.date(2016, 4, 5)), ('dangjin', 'ta', datetime.date(2016, 4, 6)), ('dangjin', 'ta', datetime.date(2016, 4, 7)), ('dangjin', 'ta', datetime.date(2016, 4, 8)), ('dangjin', 'ta', datetime.date(2016, 4, 9)), ('dangjin', 'ta', datetime.date(2016, 4, 10)), ('dangjin', 'ta', datetime.date(2016, 4, 11)), ('dangjin', 'ta', datetime.date(2016, 4, 12)), ('dangjin', 'ta', datetime.date(2016, 4, 13)), ('dangjin', 'ta', datetime.date(2016, 4, 14)), ('dangjin', 'ta', datetime.date(2016, 4, 15)), ('dangjin', 'ta', datetime.date(2016, 4, 16)), ('dangjin', 'ta', datetime.date(2016, 4, 17)), ('dangjin', 'ta', datetime.date(2016, 4, 18)), ('dangjin', 'ta', datetime.date(2016, 4, 19)), ('dangjin', 'ta', datetime.date(2016, 4, 20)), ('dangjin', 'ta', datetime.date(2016, 4, 21)), ('dangjin',

In [ ]:
# weather 지역에 따라 사용할 수 있는 변수목록을 만든다.
weather_name = IPO_DICT_READ_ME.columns[1:]
weather_cnt = [4, 5, 11, 4, 4]
weather_dict = dict()
idx = 0
for name in weather_name:
  weather_dict[name] = IPO_DICT_READ_ME[name][:weather_cnt[idx]]
  idx += 1
weather_dict.keys()

dict_keys(['dangjin', 'sinpyeong', 'seosan', 'songsan', 'songak'])

In [ ]:
weather_dict["seosan"]

0         ta
1         hm
2         ws
3         rn
4        dew
5         sp
6         lp
7         ap
8         sr
9         ss
10    ground
Name: seosan, dtype: object

## 3. clustering feature 추가하기

### 1. Add cluster In Train Data

#### 1. Trend Cluster (그래프 개형 클러스터) - 태형 버전

In [ ]:
# 서산의 습도 온도 지면온도에 대한 trend 클러스터링만 진행함
trend_df = 0
for where in [weather_name[2]]:
  for target in ['ta', 'hm', 'ground']:
    trend = trend_fn(where, target)
    if type(trend_df) == int:
      trend_df = trend
    else:
      trend_df = pd.merge(trend_df, trend[['date', where+'-'+target+'-trend']], how = 'left', on=['date'])
    data_train = pd.merge(data_train, trend[['date', where+'-'+target+'-trend']], how = 'left', on=['date'])
data_train

seosan ta 2017-01-01
seosan ta 2018-01-01
seosan ta 2019-01-01
seosan ta 2020-01-01
seosan hm 2017-01-01
seosan hm 2018-01-01
seosan hm 2019-01-01
seosan hm 2020-01-01
seosan ground 2017-01-01
seosan ground 2018-01-01
seosan ground 2019-01-01
seosan ground 2020-01-01


,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4


#### 2. Point Cluster (계절 클러스터)

In [ ]:
# point_fn("seosan", "ta")

In [ ]:
# 서산의 습도 온도 지면온도에 대한 point 클러스터링만 진행함
point_df = 0
for where in [weather_name[2]]:
  for target in ['ta', 'hm', 'ground']:
    point = point_fn(where, target)
    if type(point_df) == int:
      point_df = point
    else:
      point_df = pd.merge(point_df, point[['date', where+'-'+target+'-point']], how = 'left', on=['date'])
    data_train = pd.merge(data_train, point[['date', where+'-'+target+'-point']], how = 'left', on=['date'])
data_train

seosan ta 2017-01-01
seosan ta 2018-01-01
seosan ta 2019-01-01
seosan ta 2020-01-01
explained variance ratio : [9.61933013e-01 2.61640041e-02 5.27671126e-03 2.35069307e-03
 1.38539321e-03 8.61139159e-04 3.88140920e-04 3.16735824e-04
 2.18220404e-04 1.68175648e-04 1.28579469e-04 1.10568529e-04
 9.48391914e-05 8.30949810e-05 7.05997867e-05 6.15675304e-05
 6.12364636e-05 5.71471817e-05 5.22931727e-05 4.81422140e-05
 4.73393879e-05 4.35250150e-05 4.23884400e-05 3.64525024e-05]
seosan hm 2017-01-01
seosan hm 2018-01-01
seosan hm 2019-01-01
seosan hm 2020-01-01
explained variance ratio : [0.61171987 0.1990587  0.07231082 0.0421034  0.01903469 0.01613821
 0.00770104 0.00547802 0.00423432 0.00330066 0.00263801 0.00233716
 0.00191156 0.00159247 0.00153558 0.00134816 0.00116187 0.0010982
 0.00107642 0.00096323 0.00091997 0.00084352 0.0007888  0.00070534]
seosan ground 2017-01-01
seosan ground 2018-01-01
seosan ground 2019-01-01
seosan ground 2020-01-01
explained variance ratio : [9.35492758e-01 

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1


### 2. Add cluster In Test Data

#### 1. Trend Cluster (그래프 개형 클러스터) - 태형 버전

In [ ]:
for where in [weather_name[2]]:
  for target in ['ta', 'hm', "ground"]:
    data_test = pd.merge(data_test, trend_df[['date', where+'-'+target+'-trend']], how = 'left', on=['date'])
#data = pd.merge(data, trend_df[['date', 'bias']], how = 'left', on=['date'])
data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3


#### 2. Point Cluster (계절 클러스터)

In [ ]:
for where in [weather_name[2]]:
  for target in ['ta', 'hm', "ground"]:
    data_test = pd.merge(data_test, point_df[['date', where+'-'+target+'-point']], how = 'left', on=['date'])
#data = pd.merge(data, point_df[['date', 'bias']], how = 'left', on=['date'])
data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3,4,1,1
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3,4,1,1
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3,4,1,1
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3,4,1,1
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3,4,1,4
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3,4,1,4
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3,4,1,4
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3,4,1,4


## 일교차 추가하기

### 1. 사용하는 함수

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.externals import joblib

def changed_fn(df, where, target):
  colList = ["date", f'{where}-{target}-changed']
  changed_df = pd.DataFrame(columns=colList)

  date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date = date_start

  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print(where, target, date)
    
    temp_df = pd.DataFrame(columns=colList)
    temp_df.loc[0] = 0
    temp_df['date'] = date

    # 보간함수 불러오기
    key = (where, target, date)
    ipo = ipo_dict[key]

    # Max, Min column 넣어주기
    x = np.linspace(0, 23 * 60, 1000)
    y = spi.splev(x, ipo)
    # min_arg = x[np.argmin(y)]
    # max_arg = x[np.argmax(y)]
    min_value = np.min(y)
    max_value = np.max(y)
    temp_df[f'{where}-{target}-changed'] = (max_value - min_value)
    
    # 만들어진 df 합치기
    changed_df = changed_df.append(temp_df)
    date += timedelta(days=1)

  changed_df = changed_df.reset_index()
  ret_df = pd.merge(df, changed_df, how="left", on = ["date"])
  return ret_df

### 2. 일교차 feature 추가하기 (Train)

In [ ]:
# 서산의 습도, 온도, 지면온도에 대한 일교차를 넣어줌
data_train = changed_fn(data_train, "seosan", "ta")
data_train = changed_fn(data_train, "seosan", "hm")
data_train = changed_fn(data_train, "seosan", "ground")
data_train

seosan ta 2017-01-01
seosan ta 2018-01-01
seosan ta 2019-01-01
seosan ta 2020-01-01
seosan hm 2017-01-01
seosan hm 2018-01-01
seosan hm 2019-01-01
seosan hm 2020-01-01
seosan ground 2017-01-01
seosan ground 2018-01-01
seosan ground 2019-01-01
seosan ground 2020-01-01


,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,index_x,seosan-ta-changed,index_y,seosan-hm-changed,index,seosan-ground-changed
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,0,18.696296,0,56.993994,0,44.577377
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,0,18.696296,0,56.993994,0,44.577377
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,0,18.696296,0,56.993994,0,44.577377
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0,0,18.696296,0,56.993994,0,44.577377
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0,0,18.696296,0,56.993994,0,44.577377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1,0,5.397598,0,46.997998,0,17.498498
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1,0,5.397598,0,46.997998,0,17.498498
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1,0,5.397598,0,46.997998,0,17.498498
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1,0,5.397598,0,46.997998,0,17.498498


In [ ]:
data_train = data_train.drop(["index", "index_x", "index_y"], axis=1)

In [ ]:
data_train

,mea_ddhr,plant,loc,tem_in,hum_in,tem_coil,tem_out_loc,hum_out_loc,cond_loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed
0,2016-04-01 00:00:00,1.0,1.0,16.00,24.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377
1,2016-04-01 00:00:00,1.0,2.0,14.00,23.00,11.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377
2,2016-04-01 00:00:00,1.0,3.0,13.00,32.00,10.00,9.00,42.00,0.0,2016-04-01,0.0,6.800000,41.20,0.300000,0.0,7.2,0.900000,0.0,1013.000000,1015.700000,8.800000,41.000000,1.500000,0.0,-3.700000,4.600000,1012.100000,1015.600000,0.0,0.0,4.000000,14.00,40.000000,0.600000,0.0,13.800000,38.000000,0.400000,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377
3,2016-04-01 03:00:00,1.0,1.0,14.00,28.00,10.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377
4,2016-04-01 03:00:00,1.0,2.0,12.00,32.00,9.00,7.00,59.00,0.0,2016-04-01,180.0,5.200000,52.10,0.200000,0.0,4.7,0.300000,0.0,1012.200000,1014.900000,7.500000,40.000000,0.400000,0.0,-5.200000,4.200000,1011.400000,1014.900000,0.0,0.0,2.800000,19.60,24.000000,0.700000,0.0,20.200001,22.000000,1.000000,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350361,2019-03-31 23:30:00,2.0,2.0,9.05,38.13,9.76,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1,5.397598,46.997998,17.498498
350362,2019-03-31 23:30:00,2.0,3.0,8.32,39.13,9.49,4.57,51.47,0.0,2019-03-31,1410.0,3.000000,51.95,0.500000,0.0,2.8,2.100000,0.0,1021.150000,1023.650000,3.200000,58.500000,1.900000,0.0,-4.100000,4.550000,1020.850000,1024.050000,0.0,0.0,2.000000,8.15,24.500000,4.150000,0.0,8.150000,24.500000,4.150000,0.0,3,2,4,4,4,1,5.397598,46.997998,17.498498
350363,2019-03-31 23:40:00,2.0,1.0,8.99,37.33,9.15,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1,5.397598,46.997998,17.498498
350364,2019-03-31 23:40:00,2.0,2.0,9.02,38.49,9.68,4.47,53.14,0.0,2019-03-31,1420.0,2.933333,52.30,0.366667,0.0,2.8,2.133333,0.0,1021.166667,1023.666667,3.133333,59.666667,1.666667,0.0,-3.866667,4.633333,1020.866667,1024.066667,0.0,0.0,1.966667,8.50,18.666667,4.433333,0.0,8.500000,18.666667,4.433333,0.0,3,2,4,4,4,1,5.397598,46.997998,17.498498


### 3. 일교차 feature 추가하기 (Test)

In [ ]:
# 서산의 습도, 온도, 지면온도에 대한 일교차를 넣어줌
data_test = changed_fn(data_test, "seosan", "ta")
data_test = changed_fn(data_test, "seosan", "hm")
data_test = changed_fn(data_test, "seosan", "ground")
data_test

seosan ta 2017-01-01
seosan ta 2018-01-01
seosan ta 2019-01-01
seosan ta 2020-01-01
seosan hm 2017-01-01
seosan hm 2018-01-01
seosan hm 2019-01-01
seosan hm 2020-01-01
seosan ground 2017-01-01
seosan ground 2018-01-01
seosan ground 2019-01-01
seosan ground 2020-01-01


,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,index_x,seosan-ta-changed,index_y,seosan-hm-changed,index,seosan-ground-changed
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3,4,1,1,0,14.797798,0,66.997998,0,25.195996
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3,4,1,1,0,14.797798,0,66.997998,0,25.195996
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3,4,1,1,0,14.797798,0,66.997998,0,25.195996
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3,4,1,1,0,14.797798,0,66.997998,0,25.195996
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3,4,1,1,0,14.797798,0,66.997998,0,25.195996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3,4,1,4,0,17.982983,0,67.942943,0,29.674775
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3,4,1,4,0,17.982983,0,67.942943,0,29.674775
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3,4,1,4,0,17.982983,0,67.942943,0,29.674775
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3,4,1,4,0,17.982983,0,67.942943,0,29.674775


In [ ]:
data_test = data_test.drop(["index", "index_x", "index_y"], axis=1)

In [ ]:
data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775


## 4. 데이터 & 클러스터 모델 저장하기

In [ ]:
# 파일 저장하기
data_train.to_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature_add_cluster.csv"))
data_test.to_csv(os.path.join(DATA_CONV_PATH, "plant_test_all_weather_feature_add_cluster.csv"))

In [ ]:
# cluster trend model 저장
# joblib.dump(cluster_trend_dict, os.path.join(MODEL_PATH,'kmeans_cluster_trend_model.pkl'))

In [ ]:
# cluster point model 저장
joblib.dump(cluster_point_dict, os.path.join(MODEL_PATH,'kmeans_cluster_point_model.pkl'))

['/content/gdrive/My Drive/weather_bigcontest/4_code/model/kmeans_cluster_point_model.pkl']

In [ ]:
# i = joblib.load(os.path.join(MODEL_PATH, "kmeans_cluster_trend_model.pkl"))
# i.keys()

In [ ]:
cluster_trend_dict.keys()

dict_keys([('seosan', 'ta'), ('seosan', 'hm'), ('seosan', 'ground')])

In [ ]:
cluster_point_dict.keys()

dict_keys([('seosan', 'ta'), ('seosan', 'hm'), ('seosan', 'ground')])